# initialisation

## truc

In [1]:
'''
Import des librairies nécéssaires pour faire tourner le code
---------------------
librairies externes
---------------------
si ça ne tourne pas, c'est qu'il faut installer le package en tapant dans la console (Ctrl+ù sur VScode):

conda install package

'''
import pandas as pd
import geopandas as gpd
import os
from datetime import datetime
# from time import sleep
# from openpyxl import Workbook
# from openpyxl import load_workbook
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import Draw


'''
Libraires internes
---------------------
Si ça ne tourne pas il faut ajouter au début de votre code:

import sys
sys.path.append(r'chemin d'accès au dossier contenant la bibliothèque interne')
---------------------
Exemple:
> from library.PY_lib.coordinates import *

le chemin d'accès du dossier "library" est D:\Programmes\E.nova\python\PyEnova\flight_track\library

il faut donc ajouter au dessus :

import sys
sys.path.append(r'D:\Programmes\E.nova\python\PyEnova\flight_track')

L'opération n'est en principe a faire qu'une seule fois, tant que vos bibliothèques sont rangées au même endroit, 
d'où l'importance de ne pas éparpiller les dossiers de librairies un peu partout
'''
# import sys
# sys.path.append(r'D:\Programmes\E.nova\python\PyEnova\flight_track')

from PY_lib.data_retriever import *


## Inputs
je rajoute du texte

In [2]:
#Save folders
input_folder = r'Data\input'
output_folder = r'Data\output'

In [3]:
from pyproj import CRS, Transformer
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from geopandas.datasets import get_path
#Zone definition

# Zone = 'circle'
# Zone = 'polygon'
Zone = 'region'

Coordinates = ((0,40))
# Coordinates = ((0,40), (0,45), (5,45),(5,40))
'''longitude, latitude in WGS 84'''
Radius = 100000.0         
'''radius in meters'''
Region = 'France'       #marche pas encore
'''Region, country, city name, it may not be in database'''

current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
'''date d'extraction des données'''

"date d'extraction des données"

In [4]:
# provinces = gpd.read_file(r'Data\Geo_Data\provinces\ne_50m_admin_1_states_provinces.shp')
# display(provinces[['name','name_fr','sov_a3','adm1_code', 'geometry']])

# Country = gpd.read_file(r'Data\Geo_Data\countries\ne_110m_admin_0_countries.shp')
# display(Country[['NAME','NAME_FR','ISO_A3','FORMAL_EN','geometry']])

# City = gpd.read_file(r'Data\Geo_Data\city\ne_50m_populated_places.shp')
# display(City[['NAME','NAME_FR','SOV_A3','ADM1NAME','geometry']])

# # a=pd.DataFrame(world)
# # world.loc[world['name']==Region]
# # world.loc[world['name']==Region, 'geometry']

In [5]:
match Zone :
    case 'circle':
        # Azimuthal equidistant projection
        aeqd_proj = CRS.from_proj4(f"+proj=aeqd +lat_0={Coordinates[1]} +lon_0={Coordinates[0]} +x_0=0 +y_0=0")
        tfmr = Transformer.from_proj(aeqd_proj, aeqd_proj.geodetic_crs)
        buf = Point(Coordinates).buffer(Radius)  # distance in miles
        # area = gpd.GeoSeries(Polygon(transform(tfmr.transform, buf).exterior.coords[:]), crs='EPSG:4326')
        area = Polygon(transform(tfmr.transform, buf).exterior.coords[:])

    case 'polygon':
        # area = gpd.GeoSeries(Polygon(Coordinates), crs='EPSG:4326')
        area = Polygon(Coordinates)

    case 'region' :
        world = gpd.read_file(r'Data\Geo_Data\countries\ne_110m_admin_0_countries.shp')
        poly=world.loc[world['NAME'] == Region, 'geometry']
        # area= gpd.GeoSeries(poly, crs='EPSG:4326')
        area= poly.unary_union
        
    case _:
        raise SystemExit('undefined area')
    
gpd.GeoSeries(area, crs='EPSG:4326').explore()

In [6]:


'''
Paramètres pour délimiter la zone
'''
choice = 1
'''1=quadrilatère, 2 = circulaire'''
taille_zone = 100
'''en km si choix 2'''



'en km si choix 2'

In [7]:
Data_generation = False
'''Définir True pour faire une requete API lors de l'exectution de ce notebook,
cette option est surtout utile en mode démo et/ou pour le developpeur'''

if Data_generation:
    get_data(1, 0, 3600)

## Filtre

In [8]:
def filter_data(filename: os.path, area: gpd.GeoSeries):
    """Extract flights of input file within input area and returns a dataframe

    Args:
        filename (os.path): file path
        area (gpd.GeoSeries): Area of interest
    """
    data = pd.read_excel(filename)
    data = pd.DataFrame(data)
    
    pos=list(range(data.shape[0]))
    for index, row in data.iterrows():
        lat = row['lat']
        lng = row['lng']
        
        pos[index]=(Point(lng,lat))

    a=area.intersects(gpd.GeoSeries(pos,crs='EPSG:4326'))

    filtered_data=data[a==True]

    return filtered_data.dropna()

# Post-traitement

In [13]:
'''
Fonction qui charge les données dans le notebook pour postraitement 
et les sauvegardent dans un excel dans le dossier output
'''
round_path = r'round_coordinates.txt'
square_path = r'square_coordinates.txt'

r_latitudes, r_longitude = read_coordinates(round_path)
s_latitude, s_longitude = read_coordinates(square_path)

filename = os.listdir(input_folder)

df=[]
for idf, fname in enumerate(filename):
    path2 = os.path.join(input_folder,fname)
    filtered_data=filter_data(path2,area)
    
    # prefix = 'excel_'
    prefix = ''
    suffix = '_flight'
    date_str=''
    idx1 = fname.index(prefix)
    idx2 = fname.index(suffix)
    for idx in range(idx1 +len(prefix)+1,idx2):
        date_str=date_str+fname[idx]
    filtered_data['Date']=date_str
    
    df.append(filtered_data)


new_filename = os.path.join(output_folder,f'{current_time}_sorted_.xlsx')

filtered_df = pd.concat([*df]) 

filtered_df.to_excel(new_filename, index=False)

print(f"Tout a été enregistré dans le fichier : {new_filename}")

Tout a été enregistré dans le fichier : Data\output\2024-02-09_17-29-41_square_sorted_.xlsx


In [14]:
'''
On a ici filtered_df qui est notre dataFrame de base avec toutes nos données, les calculs sont faits, il ne reste que le postraitement.

Pour couvrir les différents cas sur la carte, on a besoin de différencier les avions qui n'apparaissent qu'une seule fois de ceux qui apparaissent plusieurs fois, 
car on ne les traite pas de la même manière:

Avions uniques : position sur la carte, flèche orientée selon leur cap
Avions qui ont plusieurs occurences: Pareil sauf que les occurences sont reliées entre elles

J'ai donc créé 2 plus petits dataframes:
Avions uniques : df_d
'''
idx=filtered_df.set_index(['hex']).index.drop_duplicates(keep=False)  
'''on créée une liste d'index (ici la colone hex), et on supprime toutes les lignes qui ont plus d'une occurence'''
df_d=filtered_df.set_index(['hex']).loc[idx].sort_index()
'''on filtre le dataframe de base avec seulement les index uniques'''
# df_d=df_d.set_index(['hex']).sort_index()
'''On définit les colones hex et date comme index, pour la lisibilité'''

df_p=filtered_df.set_index(['hex']).drop(index=idx)
'''on supprime les lignes avec seulement les index uniques, 
donc on ne garde que les lignes avec des index en double, triple, etc'''
df_p=df_p.reset_index().set_index(['hex', 'Date']).sort_index()
'''On définit les colones hex et date comme index, pour la lisibilité,
et c'est là qu'avoir du multiindex prend tout son sens.'''



"On définit les colones hex et date comme index, pour la lisibilité,\net c'est là qu'avoir du multiindex prend tout son sens."

In [15]:
'''
Affichage des deux dataframes:
'''
# display(df_d)
# display(df_p)

,reg_number,flag,lat,lng,alt,dir,speed,v_speed,squawk,flight_number,...,dep_icao,dep_iata,arr_icao,arr_iata,airline_icao,airline_iata,aircraft_icao,updated,status,Date
hex,,,,,,,,,,,,,,,,,,,,,
0200EB,CN-RGF,MA,48.639640,2.086896,2514.0,270.8,503.0,13.3,7622.0,777Z,...,LFPO,ORY,GMMN,CMN,RAM,AT,B738,1697787940,en-route,023-10-20_09-45-52
0200F8,CN-RGJ,MA,46.917347,-1.782673,11887.0,217.4,822.0,0.0,7611.0,747,...,LFPO,ORY,GMMX,RAK,RAM,AT,B738,1697787940,en-route,023-10-20_09-45-52
02A191,TS-IMR,TN,45.226709,6.064453,11239.0,339.8,963.0,4.9,1000.0,634,...,DTTJ,DJE,LFPG,CDG,TAR,TU,A320,1697787940,en-route,023-10-20_09-45-52
02A198,TS-IMY,TN,43.590398,5.130550,1950.0,2.1,452.0,-4.9,1000.0,930,...,DTTA,TUN,LFML,MRS,TAR,TU,A20N,1697787940,en-route,023-10-20_09-45-52
02A1A2,TS-INC,TN,47.868118,4.194168,7101.0,335.5,864.0,-12.7,1000.0,542,...,DTMB,MIR,LFPG,CDG,LBT,BJ,A320,1697787940,en-route,023-10-20_09-45-52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C023B7,C-FNNQ,CA,49.671295,-1.789470,10896.0,97.0,917.0,-7.5,6304.0,870,...,CYUL,YUL,LFPG,CDG,ACA,AC,B77W,1697787940,en-route,023-10-20_09-45-52
C02EE7,C-FRTU,CA,45.257538,6.103167,12496.0,112.0,901.0,-0.3,1164.0,890,...,CYYZ,YYZ,LIRF,FCO,ACA,AC,B789,1697787940,en-route,023-10-20_09-45-52
C04FBB,C-GEFA,CA,44.272156,2.986473,11193.0,105.0,874.0,-4.6,1000.0,814,...,CYUL,YUL,LFMN,NCE,ACA,AC,A333,1697787940,en-route,023-10-20_09-45-52


reg_number flag        lat       lng      alt  \
hex    Date                                                               
02007A 023-10-06_18-15-49     CN-ROP   MA  43.895279  1.972174  12001.0   
       023-10-06_18-18-51     CN-ROP   MA  44.059831  2.442758  10660.0   
02008B 023-10-06_18-15-49     CN-ROS   MA  44.514679 -1.302468  10972.0   
       023-10-06_18-18-51     CN-ROS   MA  44.847931 -1.046252  10972.0   
020092 023-10-06_18-15-49     CN-ROT   MA  45.812347  5.987000   6957.0   
...                              ...  ...        ...       ...      ...   
89631D 023-10-06_18-18-51     A6-EFM   AE  48.624573  5.833881  11277.0   
8963F2 023-10-06_18-15-49     A6-EOU   AE  48.555038  2.276987  11582.0   
       023-10-06_18-18-51     A6-EOU   AE  48.663986  1.723656  11582.0   
AC1E56 023-10-06_18-15-49     N880FD   US  48.408834  3.914747   4244.0   
       023-10-06_18-18-51     N880FD   US  48.594543  3.602600   3345.0   

                             dir  speed  v_speed  squawk flight_number  ...  \
hex    Date                                                             ...   
02007A 023-10-06_18-15-49   63.0  850.0     -6.2  1000.0          720P  ...   
       023-10-06_18-18-51   64.0  855.0    -10.1  1000.0          720P  ...   
02008B 023-10-06_18-15-49   28.0  859.0      0.3  1000.0          750P  ...   
       023-10-06_18-18-51   28.0  854.0      0.0  1000.0          750P  ...   
020092 023-10-06_18-15-49  185.7  853.0      7.8  5704.0          931G  ...   
...                          ...    ...      ...     ...           ...  ...   
89631D 023-10-06_18-18-51  101.2  931.0      0.0   647.0          9962  ...   
8963F2 023-10-06_18-15-49  286.6  866.0      0.0  5707.0           4PV  ...   
       023-10-06_18-18-51  286.3  873.0      0.0  5707.0           4PV  ...   
AC1E56 023-10-06_18-15-49  317.1  680.0     -6.2  1000.0             9  ...   
       023-10-06_18-18-51  302.8  601.0     -0.3  1000.0             9  ...   

                          flight_iata dep_icao dep_iata arr_icao arr_iata  \
hex    Date                                                                 
02007A 023-10-06_18-15-49       AT720     GMMN      CMN     LFLL      LYS   
       023-10-06_18-18-51       AT720     GMMN      CMN     LFLL      LYS   
02008B 023-10-06_18-15-49       AT750     GMMN      CMN     LFPO      ORY   
       023-10-06_18-18-51       AT750     GMMN      CMN     LFPO      ORY   
020092 023-10-06_18-15-49       AT931     LSGG      GVA     GMMN      CMN   
...                               ...      ...      ...      ...      ...   
89631D 023-10-06_18-18-51      EK9962     LFPG      CDG     OMDW      DWC   
8963F2 023-10-06_18-15-49       EK205     LIMC      MXP     KJFK      JFK   
       023-10-06_18-18-51       EK205     LIMC      MXP     KJFK      JFK   
AC1E56 023-10-06_18-15-49         FX9     OMDB      DXB     LFPG      CDG   
       023-10-06_18-18-51         FX9     OMDB      DXB     LFPG      CDG   

                          airline_icao airline_iata aircraft_icao     updated  \
hex    Date                                                                     
02007A 023-10-06_18-15-49          RAM           AT          B738  1696608995   
       023-10-06_18-18-51          RAM           AT          B738  1696609172   
02008B 023-10-06_18-15-49          RAM           AT          B738  1696608995   
       023-10-06_18-18-51          RAM           AT          B738  1696609172   
020092 023-10-06_18-15-49          RAM           AT          B738  1696608995   
...                                ...          ...           ...         ...   
89631D 023-10-06_18-18-51          UAE           EK          B77L  1696609172   
8963F2 023-10-06_18-15-49          UAE           EK          A388  1696608995   
       023-10-06_18-18-51          UAE           EK          A388  1696609172   
AC1E56 023-10-06_18-15-49          FDX           FX          B77L  1696608995   
       023-10-06_18-18-51          FDX          

# OUTPUTS

## map folium

In [16]:
'''
Carte avec les avions affichés, après pas mal d'essais sur plotly, je suis finalement passé sur folium
J'ai aussi reçu un coup de main de chatGPT, dont j'ai tout de même du adapter le code pour que ça marche, 
j'ai laissé ses commentaires.
'''
# Create a Folium Map
m = folium.Map(location=[df_p['lat'].mean(), df_p['lng'].mean()], zoom_start=6)

# Iterate through your data and create lines and markers
for i in df_p.index.get_level_values(0):
    locations = []  # Accumulate coordinates for each i    
    for j in df_p.loc[i].index.get_level_values(0):
        angle = df_p.loc[(i,j), 'dir']  # Angle in degrees
        # Create a marker
        marker = folium.Marker(
            location=(df_p.loc[(i,j), 'lat'], df_p.loc[(i,j), 'lng']),
            icon=folium.DivIcon(
                html=f'<div><div style="transform: rotate({angle}deg); border: 8px solid transparent; border-bottom-color: blue; width: 0; height: 0;"></div></div>'
            ),
            tooltip=i
        )

        m.add_child(marker)

        # Accumulate coordinates for the line
        locations.append((df_p.loc[(i,j), 'lat'], df_p.loc[(i,j), 'lng']))
    
    # Create a line (PolyLine)
    line = folium.PolyLine(
        locations=locations,
        color='blue',
        weight=2,
        opacity=1,
        tooltip=i
    )
    
    m.add_child(line)

for i in df_d.index:
    angle = df_d.loc[i, 'dir']  # Angle in degrees
    # Create a marker
    marker = folium.Marker(
        location=(df_d.loc[i, 'lat'], df_d.loc[i, 'lng']),
        icon=folium.DivIcon(
            html=f'<div><div style="transform: rotate({angle}deg); border: 8px solid transparent; border-bottom-color: blue; width: 0; height: 0;"></div></div>'
        ),
        tooltip=i
    )

    m.add_child(marker)


# Add the drawing tools for interactive features (optional)
draw = Draw(export=True)
draw.add_to(m)

# Display the map

# m.save(r'C:\Users\e.NOVA Aerospace\Documents\PYTHON\flight_track\Data\output\folium_map.html')
m.save(os.path.join(output_folder,f'folium_map.html'))
m


In [17]:
'''
Tentative 2 avec plotly que je laisse pour la beauté de la chose, mais j'arrivais pas à afficher une flèche orientée selon le cap
'''
fig = go.Figure()

for i in df_p.index.get_level_values(0):
    fig.add_trace(go.Scattermapbox(
        mode="lines+markers", #
        lon=df_p.loc[i, 'lng'],
        lat=df_p.loc[i, 'lat'],
        name=i, 
        marker=dict(
            symbol="arrow",
            size=10,
            angle=df_p.loc[i, 'dir'],
            ),
        ))
    


fig.update_layout(
    margin={'l': 113, 't': 24, 'b': 22, 'r': 115},
    mapbox=dict(style='carto-positron',
                center=go.layout.mapbox.Center(lon=7, lat=43.5),
                pitch=0,
                zoom=6)
)

fig.show()

In [19]:
'''
Exemple de graph très rapidement sortable avec plotly
On peut cliquer sur la légende à droite pour afficher/cacher des objets
'''
fig = px.scatter(
    data_frame=filtered_df,
    y='alt',
    x='speed',
    color='Date'
)
fig.show()

## Nombre avion par date

In [20]:
'''
comptage du nombre d'avions par date
'''
dff=pd.DataFrame(index=filtered_df['Date'].unique(), columns=['Nombre'])
for i in filtered_df['Date'].unique():
    dff.loc[i, 'Nombre']=filtered_df['Date'].where(filtered_df['Date']==i).count()

'''
affichage avec px.
'''
fig = px.bar(
    data_frame=dff,
    y='Nombre',
    x=dff.index,
    # color='Date'
)
fig.show()

# fig.write_html(r'C:\Users\e.NOVA Aerospace\Documents\PYTHON\flight_track\output\nb_avion.html')
fig.write_html(os.path.join(output_folder,f'nb_avion.html'))

In [ ]:
filtered_df.columns

Index(['hex', 'reg_number', 'flag', 'lat', 'lng', 'alt', 'dir', 'speed',
       'v_speed', 'squawk', 'flight_number', 'flight_icao', 'flight_iata',
       'dep_icao', 'dep_iata', 'arr_icao', 'arr_iata', 'airline_icao',
       'airline_iata', 'aircraft_icao', 'updated', 'status', 'Date'],
      dtype='object')

In [ ]:
df_airline=pd.DataFrame(index=filtered_df['arr_iata'].unique(), columns=['Nombre'])
for i in filtered_df['arr_iata'].unique():
    df_airline.loc[i, 'Nombre']=filtered_df['arr_iata'].where(filtered_df['arr_iata']==i).count()

'''
Exemple de graph très rapidement sortable avec plotly
On peut cliquer sur la légende à droite pour afficher/cacher des objets
'''
fig = px.bar(
    data_frame=df_airline,
    y='Nombre',
    x=df_airline.index,
    # color=df_airline.index
)
fig.show()

In [ ]:
df_aircraft=pd.DataFrame(index=filtered_df['aircraft_icao'].unique(), columns=['Nombre'])
for i in filtered_df['aircraft_icao'].unique():
    df_aircraft.loc[i, 'Nombre']=filtered_df['aircraft_icao'].where(filtered_df['aircraft_icao']==i).count()
    df_aircraft=df_aircraft.sort_index()
'''
Exemple de graph très rapidement sortable avec plotly
On peut cliquer sur la légende à droite pour afficher/cacher des objets
'''
fig = px.bar(
    data_frame=df_aircraft,
    y='Nombre',
    x=df_aircraft.index,
    title="type d'avion"

)
fig.show()